# Set the Database in NEO4J
Get all the pre_existant data and store is to a Neo4j dataset.<br />
Will be ultimatly divide between :
- a .py file peer pre-existante dataset to scrap all the JSON
- neo4j.py : take the scraped JSON file and send it to NEO4J

In [42]:
#!pip install selenium
#!pip install neo4j

In [36]:
# Selenium
from selenium import webdriver
from selenium.webdriver.edge.options import Options
import requests

# Convert
import json
import os

# NEO4J
from neo4j import GraphDatabase

# Studium
The University Paris 1 provide a dictionnary of Middle Age personnality called [Studium](http://studium-parisiense.univ-paris1.fr/?action=index) with 12.446 nammed entities.<br />
We scrap the site with Selenium and import the data in a pandas.DataFrame

In [3]:
options = webdriver.EdgeOptions()
options.add_argument("--headless=new")
driver = webdriver.Edge(options=options)

radical = "http://studium-parisiense.univ-paris1.fr/?action=index&letter="
get_href = lambda card : card.get_attribute("href")

In [4]:
all_entites_links = []

# Start from Y as an exemple because to many data for my computer
for letter in range(ord('Y'), ord('Z') + 1):
    # Go on the subpage of each lettre
    url = radical + chr(letter)
    driver.get(url)
    print(f"Connection to {chr(letter)} : {url}")
    
    # Extract all the link associate to an entite
    entite_cards = driver.find_elements(by='css selector', value='h5.card-title a')
    entites_links = list(map(get_href, entite_cards))
    [print(link) for link in entites_links]
    print()
    
    # Add the list to more global one
    all_entites_links.extend(entites_links)

Connection to Y : http://studium-parisiense.univ-paris1.fr/?action=index&letter=Y
http://studium-parisiense.univ-paris1.fr/individus/56193-yehieldeparis
http://studium-parisiense.univ-paris1.fr/individus/21406-ymbaudusplantelin
http://studium-parisiense.univ-paris1.fr/individus/12711-yoniusgazeau

Connection to Z : http://studium-parisiense.univ-paris1.fr/?action=index&letter=Z
http://studium-parisiense.univ-paris1.fr/individus/52335-zachariasdebureto
http://studium-parisiense.univ-paris1.fr/individus/12438-zachariasdechebsey
http://studium-parisiense.univ-paris1.fr/individus/21425-zachariasdelamote
http://studium-parisiense.univ-paris1.fr/individus/12437-zachariasdeparma
http://studium-parisiense.univ-paris1.fr/individus/12440-zebrandusdedelf
http://studium-parisiense.univ-paris1.fr/individus/12442-zenopetridecampis
http://studium-parisiense.univ-paris1.fr/individus/51948-zenodecastellione
http://studium-parisiense.univ-paris1.fr/individus/56192-zenobiusbonacciideguasconibus
http://st

In [18]:
entities= []

for entity_link in all_entites_links:
    # Go on the subpage of each entities
    driver.get(entity_link)
    
    # Extract all the link associate to an json file
    json_button = driver.find_element(by='css selector', value='div.app-page a.btn.btn-success')
    json_link = get_href(json_button)

    # Request the json file
    response = requests.get(json_link)
    response.raise_for_status()
    
    # Store all the json on a Json list
    entite = json.loads(response.content)
    print(json.dumps(entite))
    entities.append(entite)

{"_id": "6486bde53e5d2d3083b2fb0e", "reference": "56193", "identity": {"name": [{"value": "YEHIEL de Paris", "meta": {}}], "nameVariant": [{"value": "$Yehiel de PARIS$", "meta": {"names": ["Yehiel de PARIS"]}}, {"value": "$Yehiel BEN JOSEPH$", "meta": {"names": ["Yehiel BEN JOSEPH"]}}, {"value": "$Vives MELDENSIS$", "meta": {"names": ["Vives MELDENSIS"]}}], "shortDescription": [{"value": "Rabbin", "meta": {}}], "datesOfLife": [{"value": "%    -1268%", "meta": {"dates": []}, "comment": ["\t/Il serait mort \u00e0 *Haifa en %1268% ?"]}], "datesOfActivity": [{"value": "%1218-1268%", "meta": {"dates": [{"type": "INTERVAL", "startDate": {"type": "SIMPLE", "date": 1218}, "endDate": {"type": "SIMPLE", "date": 1268}}]}}], "activityMediane": [{"value": "%1243%.", "meta": {"dates": [{"type": "SIMPLE", "date": 1243}]}}], "gender": [{"value": "male", "meta": {"isComment": false, "isLink": false}}], "status": [{"value": "Ext\u00e9rieur", "meta": {}}]}, "origin": {"birthPlace": [{"value": "Ile-de-Fra

# Storing

In [38]:
folder_path = ".\data"

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print("Data folder created.")
else:
    print("Data folder already exists.")

Data folder already exists.


In [39]:
for entity in entities:
    # Write a unique file name
    id_name = entity["identity"]["name"][0]["value"]
    id_name = id_name.replace(" ", "-") # Avoid issue with space in file name
    reference = entity["reference"]     # In case same name & make easy Ctrl+F
    file_name = f"./data/{id_name}_{reference}.json" 
    
    # Convert the json in string, remove ascii to allowed UTF-8, put indent to easy human reading
    with open(file_name, "w", encoding='utf-8') as file:
        json.dump(entity, file, indent=2, ensure_ascii=False)
    
    print(file_name) 

./data/YEHIEL-de-Paris_56193.json
./data/YMBAUDUS-Plantelin_21406.json
./data/YONIUS-Gazeau_12711.json
./data/ZACHARIAS-de-Bureto_52335.json
./data/ZACHARIAS-de-Chebsey_12438.json
./data/ZACHARIAS-de-Lamote_21425.json
./data/ZACHARIAS-de-Parma_12437.json
./data/ZEBRANDUS-de-Delf_12440.json
./data/ZENO-Petri-de-Campis_12442.json
./data/ZENO-de-Castellione_51948.json
./data/ZENOBIUS-Bonaccii-de-Guasconibus_56192.json
./data/ZILFREDUS-de-Plaisance_12445.json
./data/ZINDURAMMUS_16899.json
./data/ZUNTA_12446.json


# Relation

In [ ]:
# extraction relation & tag

# NEOJ4

In [50]:
# From a file downloaded
NEO4J_URI="neo4j+s://6f247dde.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="uHfWHWw7lxdquz2HjbmIvsUrklk6Rw9_VRgkM15QumE"
AURA_INSTANCEID="6f247dde"
AURA_INSTANCENAME="Entities"

In [ ]:
# WRITE

In [51]:
driver = GraphDatabase.driver(uri=NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
driver.verify_connectivity()
session = driver.session()

Transferer les json vers noe4j

In [ ]:
session.close()
driver.close()